In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("titanic.csv")
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
#Xóa các giá trị tuổi outline
q25 = df['Age'].quantile(0.25)
q75 = df['Age'].quantile(0.75)
iqr = q75 - q25
lower_bound = q25 - 1.5 * iqr
upper_bound = q75 + 1.5 * iqr
df = df[(df['Age'] >= lower_bound) & (df['Age'] <= upper_bound)]

In [6]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
885    False
886    False
887    False
889    False
890    False
Length: 703, dtype: bool

In [7]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          523
Embarked         2
dtype: int64

In [8]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [9]:
df = df.drop(['PassengerId', 'Name'],axis=1)

In [10]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [11]:
#mô hình 1 là xóa các giá trị missing values
df1 = df.dropna()

In [12]:
df1.tail(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
835,1,1,female,39.0,1,1,PC 17756,83.1583,E49,C
853,1,1,female,16.0,0,1,PC 17592,39.4000,D28,S
857,1,1,male,51.0,0,0,113055,26.5500,E17,S
862,1,1,female,48.0,0,0,17466,25.9292,D17,S
867,0,1,male,31.0,0,0,PC 17590,50.4958,A24,S
871,1,1,female,47.0,1,1,11751,52.5542,D35,S
872,0,1,male,33.0,0,0,695,5.0000,B51 B53 B55,S
879,1,1,female,56.0,0,1,11767,83.1583,C50,C
887,1,1,female,19.0,0,0,112053,30.0000,B42,S
889,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [13]:
df1.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
dtype: int64

In [14]:
#mô hình 2 không xóa mà thay thế các giá trị missing value bằng giá trị phù hợp 
#(biến số thay thế = mean; biến chuổi thay thế =mode (giá trị xuất hiện nhiều nhất)
df2 = df.copy()
numeric_cols = df2.select_dtypes(include='number').columns
for col in numeric_cols:
    df2[col].fillna(df2[col].mean(), inplace=True)

categorical_cols = df2.select_dtypes(include='object').columns
for col in categorical_cols:
    mode_val = df2[col].mode()[0]  # Lấy mode (giá trị xuất hiện nhiều nhất)
    df2[col].fillna(mode_val, inplace=True)

In [15]:
df2.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,B96 B98,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,B96 B98,S
6,0,1,male,54.0,0,0,17463,51.8625,E46,S
7,0,3,male,2.0,3,1,349909,21.0750,B96 B98,S
8,1,3,female,27.0,0,2,347742,11.1333,B96 B98,S
9,1,2,female,14.0,1,0,237736,30.0708,B96 B98,C
10,1,3,female,4.0,1,1,PP 9549,16.7000,G6,S


In [16]:
df2.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
dtype: int64

In [18]:
#logistic regression mô hình 1
df_encoded = pd.get_dummies(df1)
X = df_encoded.drop('Survived', axis=1)  # Biến độc lập
y = df_encoded['Survived']  # Biến phụ thuộc

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng mô hình logistic regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá hiệu suất của mô hình
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8055555555555556


In [20]:
#logistic regression mô hình 2
df2_encoded = pd.get_dummies(df2)
X2 = df2_encoded.drop('Survived', axis=1)  # Biến độc lập
y2 = df2_encoded['Survived']  # Biến phụ thuộc

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra cho df2
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Xây dựng mô hình logistic regression cho df2
model2 = LogisticRegression(max_iter=1000)
model2.fit(X_train2, y_train2)

# Dự đoán trên tập kiểm tra cho df2
y_pred2 = model2.predict(X_test2)

# Đánh giá hiệu suất của mô hình 2
accuracy2 = accuracy_score(y_test2, y_pred2)
print("Accuracy (mô hình 2):", accuracy2)


Accuracy (mô hình 2): 0.8085106382978723
